In [2]:
import numpy as np  
import pandas as pd  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import LSTM, Dense  
from sklearn.preprocessing import MinMaxScaler
import torch  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, TensorDataset  
from sklearn.preprocessing import MinMaxScaler  
from transformers import BertTokenizer, BertModel  

E:\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# df1 = pd.read_csv('data/开发大表_修订.csv')

# chunk_size = 1000  # 设置每次读取的行数
# dataframes = pd.read_csv('data/汇总数据_修订_天数对齐.csv', chunksize=chunk_size)
# df2 = pd.concat(dataframes)

In [5]:
# # # 根据'井号'进行一对多生成新的数据  
# df = pd.merge(df2, df1, on='井号', how='inner')
# columns = df.columns
# new_columns = []
# for column in columns:
#     new_columns.append(column.replace('\n', '').replace(' ', ''))
# df.columns = new_columns
# df.to_csv('data/数据ALL.csv', index=False)

In [6]:
# chunk_size = 1000  # 设置每次读取的行数
# dataframes = pd.read_csv('data/数据ALL.csv', chunksize=chunk_size)
# df = pd.concat(dataframes)

In [7]:
# df.drop(['日产水','日外输气量','历年累计产水量','备注说明', '泡排施工日期','泡排当前日期','泡排施工前日产（万方/天）','90天累产(104m3)',
#         '泡排施工后日产（万方/天）','泡排备注','设备规格',
#          '作业井号', '施工日期', '施工开始时间', '施工结束时间',
#        '施工排量（方/小时）', '气举方式', '施工前压力（兆帕）', '施工最高压力（兆帕）',
#        '施工后压力（兆帕）', '施工前瞬量（万方/天）', '施工最高瞬产（万方/天）'
#         ,'施工后瞬量（万方/天）','注气量（万方）'
#         ,'施工前日产（万方）', '施工后日产（万方）','排液量（方）','作业时间（小时）','备注','考虑天数'
#         ,'增产气量(万方）','气举类型','施工时段','产气量','实际产气量','是否增压','增压机设备名称','增压机类型','增压机备注'
#         ], axis=1, inplace=True)

In [8]:
# # # 指定列名，将含有空值的特定列填充为0
# null_columns = df.columns[df.isna().any()].tolist()
# for null_column in null_columns:
#     df[null_column].fillna(0, inplace=True)
# df.to_csv('data/数据ALL_修订.csv', index=False)

In [9]:
chunk_size = 1000  # 设置每次读取的行数
dataframes = pd.read_csv('data/数据ALL_修订.csv', chunksize=chunk_size)
df = pd.concat(dataframes)

In [10]:
df.columns

Index(['井号', '日期', '生产时间', '套压', '油压', '日产气', '输压', '历年累计产气量', '甜点分区', '生产天数',
       '研究所复核储量丰度-2023.6.30(108m3/km2)', 'I类储层厚度(m)', 'I类储层连续厚度(m)',
       '龙一11厚度(m)', '龙一11(1类厚度)(m)', '龙一11水平长度(m)', '龙一11压力系数', '龙一11钻遇率(%)',
       '龙一11底以上4米箱体钻遇率（%）', '水平段长(m)', '主体单段簇数', '单段主体孔数', '簇间距(m)',
       '主体射孔密度(P/m)', '加砂强度(t/m)', '用液强度(m3/m)', '分段段长(m)', '是否套变', '合压长度(m)',
       '丢段长度(m)', '优质页岩钻遇率（%）', '设计压裂段长(m)', '实际射孔簇数', '折算有效段数', '100目石英砂(t)',
       '40/70目石英砂(t)', '40/70目陶粒(t)', '30/50目陶粒(t)', '总砂量(t)', '酸液(m3)',
       '滑溜水(m3)', '线性胶(m3)', '弱凝胶(m3)', '总液量(m3)', '平均单段砂量(t)', '平均单段液量(m3)',
       '陶粒比例（%）', '平均停泵压力(MPa)', '排量（m3/min）', '压裂结束停泵压力(MPa)', '焖井时间(天)',
       '开始排液时井口套压(MPa)', '投产前累计排液量(m3)', '投产前返排率(%)', '井间距(m)'],
      dtype='object')

In [11]:
df1 = df[df['井号'] == '威202H1-1'  ]
# df1.drop(['日期','日产气','甜点分区','井号'], axis=1, inplace=True)
# df1.drop(['日期','井号'], axis=1, inplace=True)
data = df1[['历年累计产气量']]

In [8]:
data = df1['历年累计产气量'].values
data = data.reshape(-1, data.shape[-1])  

# 数据归一化  
scaler = MinMaxScaler()  
data= scaler.fit_transform(data)  

print(len(data))
aa
# 创建数据集  
def create_dataset(data, seq_length):  
    X, y = [], []  
    for i in range(len(data) - seq_length):  
        X.append(data[i:(i + seq_length), :])  
        y.append(data[(i + seq_length), :])  
    return np.array(X), np.array(y)  


# 创建数据集  
seq_length = 10  # 序列长度，根据你的数据调整  
X, y = create_dataset(data, seq_length)  

print(X)
aa

# 转换为PyTorch TensorDataset  
tensor_x = torch.from_numpy(X)  
tensor_y = torch.from_numpy(y)  
dataset = TensorDataset(tensor_x, tensor_y)  
  
# 创建DataLoader  
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  
  
# 加载预训练的Transformer模型和tokenizer  
model_name = 'bert-base-uncased'  # 使用BERT作为示例，你可以选择其他预训练模型，如'bert-base-chinese'、'bert-large-cased'等  
tokenizer = BertTokenizer.from_pretrained(model_name)  
model = BertModel.from_pretrained(model_name)  
  
# 为模型的输出层添加一个新的线性层，以适应你的任务（多分类或回归）  
if len(data.shape) == 2:  # 如果数据是二维的（即，有多个特征），则使用多分类问题。假设有num_classes个类。  
    num_classes = 10  # 假设有10个类，根据你的任务调整。  
    model.classifier = Linear(model.config.hidden_size, num_classes)  
else:  # 如果数据是一维的，则使用回归问题。  
    model.classifier = Linear(model.config.hidden_size, 1)  
  
# 定义损失函数和优化器  
criterion = MSELoss()  # 使用均方误差作为损失函数，适用于回归问题。对于多分类问题，你可能想使用CrossEntropyLoss。  
optimizer = Adam(model.parameters(), lr=0.001)  # 使用Adam优化器。你可以根据你的需要更改学习率。  
  
# 训练模型  
num_epochs = 100  # 训练轮数，根据你的需要调整。  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 检查是否有可用的GPU，并将模型和数据移动到适当的设备上。  
model.to(device)  # 将模型移动到设备上。如果你的设备是CPU，则不需要此行。  
for epoch in range(num_epochs):  
    model.train()  # 设置模型为训练模式。  
    for batch in dataloader:  
        input_ids = batch[0].to(device)  # 将输入数据传递到设备（例如GPU）中。如果你的设备是CPU，则不需要此行。  
        target = batch[1].to(device)  # 将目标数据传递到设备中。如果你的设备是CPU，则不需要此行。  
        outputs = model(input_ids)  # 前向传播，得到预测输出。  
        loss = criterion(outputs, target)  # 计算损失。  
        optimizer.zero_grad()  # 清零梯度。  
        loss.backward()  # 反向传播。  
        optimizer.step()  # 更新权重。  
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")  # 打印每个epoch的损失。

1


NameError: name 'aa' is not defined

In [9]:

# 构建输入数据形状  
n_features = 1#len(featureNames)  # 单个输入特征（时间序列数据）  
n_steps = 3  # 输入序列的步长（考虑过去的X个时间步）  
X, y = [], []  
for i in range(len(data) - n_steps):  
    X.append(data['历年累计产气量'].values[i:i + n_steps])  
    y.append(data['历年累计产气量'].values[i + n_steps])  
X, y = np.array(X), np.array(y)  
X = X.reshape((X.shape[0], X.shape[1], n_features))  # 输入数据的形状：(样本数, 时间步长, 特征数)  
  
# 创建LSTM模型  
model = Sequential()  
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))  # 50个隐藏单元  
model.add(Dense(1))  # 输出层，使用线性激活函数  
model.compile(loss='mean_squared_error', optimizer='adam')  # 使用均方误差作为损失函数，使用Adam优化器进行训练  
  
# 训练模型  
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)  # 使用80%的数据进行训练，20%的数据用于验证

IndexError: tuple index out of range